In [ ]:
%pip install pandas numpy

In [1]:
import pandas as pd
import numpy as np

# Self-Attention Mechanism for One Head

Self-Attention is a mechanism introduced in the paper "Attention is All You Need" to compute relationships between elements in a sequence. For a simple vocabulary like "I am Max" with dimensions of 768, the process involves:

1. **Input Representation**: Each word is represented as a vector of size 768.
2. **Query, Key, and Value Matrices**: These (Q, K, V) are randomly initialized for the weights based on the sequence length (L) and the dimensions (768).

3. **Scaled Dot-Product Attention**:
   - Compute the dot product between the Query and Key matrices.
   - Scale the result by the square root of the dimension (768).
   - Apply a mask to ensure causality (future words do not influence past words).
   - Use the softmax function to normalize the scores.
4. **Weighted Sum**: Multiply the attention scores with the Value matrix to get the final representation.

This mechanism is important because it allows the model to focus on relevant parts of the input sequence, enabling better understanding and generation of context-aware outputs.

---

*This demonstration of how self-attention works was inspired by [this video](https://www.youtube.com/watch?v=QCJQG4DuHT0&t=3s), [this repository](https://github.com/ajhalthor/Transformer-Neural-Network), and the instructor [Ajay Halthor](https://github.com/ajhalthor).*

In [ ]:
L, d_k, d_v = 3, 768, 768

Q = np.random.rand(L, d_k)
K = np.random.rand(L, d_k)
V = np.random.rand(L, d_v)

In [19]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.443965,0.552489,0.795881,0.912014,0.927523,0.928838,0.981990,0.415161,0.434954,0.841408,...,0.664132,0.549578,0.961703,0.434516,0.254016,0.601373,0.726527,0.299726,0.975034,0.543424
1,0.147352,0.662447,0.347690,0.443408,0.526462,0.415522,0.920554,0.846130,0.011775,0.869826,...,0.067864,0.930125,0.102429,0.006717,0.744869,0.326065,0.079725,0.596883,0.053978,0.305716
2,0.134103,0.689522,0.260076,0.682714,0.257299,0.498879,0.338012,0.475656,0.439719,0.145602,...,0.032548,0.853616,0.119093,0.344746,0.136598,0.763248,0.069965,0.829748,0.818024,0.475082


In [20]:
pd.DataFrame(K)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.967386,0.641188,0.334244,0.363748,0.462085,0.423704,0.646040,0.610683,0.529257,0.408328,...,0.825135,0.812598,0.196644,0.844694,0.498152,0.800483,0.981069,0.741634,0.460788,0.702955
1,0.088451,0.325549,0.519656,0.389377,0.756308,0.851370,0.752869,0.095203,0.971188,0.700056,...,0.851865,0.328944,0.698538,0.422627,0.499308,0.908873,0.400249,0.785511,0.446014,0.601970
2,0.246552,0.821833,0.760850,0.730059,0.710080,0.369434,0.826794,0.761943,0.237624,0.233502,...,0.884720,0.888363,0.819361,0.864076,0.938706,0.635196,0.966073,0.603802,0.364685,0.609665


In [21]:
pd.DataFrame(V)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.300657,0.174889,0.550840,0.542541,0.713070,0.654922,0.014812,0.846204,0.020754,0.318203,...,0.802917,0.550429,0.412732,0.671187,0.546453,0.680948,0.906085,0.210110,0.048897,0.968737
1,0.641522,0.353871,0.485978,0.025493,0.029522,0.296605,0.304838,0.853458,0.016998,0.996952,...,0.704518,0.606242,0.363611,0.330029,0.762868,0.375294,0.550619,0.049723,0.170887,0.129178
2,0.422075,0.718450,0.209494,0.852302,0.493842,0.602548,0.880430,0.345424,0.430350,0.797527,...,0.946100,0.317563,0.387658,0.094867,0.760170,0.209812,0.134985,0.082697,0.142534,0.241684


Scaling the dot product of Q and K by $\sqrt{d_k}$ keeps the attention scores in a reasonable range, making the softmax more stable and improving training.

In [22]:
import math
scaled = np.matmul(Q, K.T) / math.sqrt(d_k)

In [23]:
pd.DataFrame(scaled)

,0,1,2
0,6.714893,6.465850,6.558512
1,6.770609,6.569006,6.534449
2,6.892509,6.548279,6.485003


In the code above, we create a **mask matrix** `M` to enforce causality in the self-attention mechanism. The mask is a lower triangular matrix where the upper diagonal elements are set to $-\infty$. This ensures that, when added to the attention scores before applying the softmax, the softmax output for those positions becomes zero. As a result, each position in the sequence can only attend to itself and previous positions, not to any future positions. This is crucial for tasks like language modeling, where future information should not be accessible.

In [27]:
M = np.tril(np.ones((L,L)))
M[M== 0] = -np.inf
M[M == 1] = 0

In [28]:
pd.DataFrame(M)

,0,1,2
0,0.0,-inf,-inf
1,0.0,0.0,-inf
2,0.0,0.0,0.0


After computing the scaled dot product of $Q$ and $K$ (i.e., $(QK^T)/\sqrt{d_k}$), we add the mask matrix $M$ to the result. This mask ensures that each position in the sequence can only attend to itself and previous positions, not future ones. The masked and scaled attention scores are then passed through the softmax function, which converts them into a probability distribution. This distribution determines how much focus (attention) each word should give to every other word in the sequence, while respecting the causality constraint imposed by the mask.

In [29]:
from scipy.special import softmax

attention = softmax(scaled + M)

After obtaining the softmax distribution (the attention weights), we multiply it by the Value matrix $V$ to produce the final output of the self-attention mechanism:

$$\text{Output} = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}} + M\right) \cdot V$$

**Why is this multiplication needed?**

- The softmax distribution tells us how much attention each word in the sequence should pay to every other word (including itself).
- By multiplying these attention weights with the Value matrix $V$, we compute a weighted sum of the value vectors for each position in the sequence.
- This means each output vector is a blend of the value vectors, where the contribution of each value is determined by the attention score.
- This allows the model to aggregate information from relevant positions in the sequence, enabling it to capture dependencies and context effectively.

In summary, this multiplication enables the self-attention mechanism to produce context-aware representations for each word, based on how much attention it gives to other words in the sequence.

In [31]:
new_V = np.matmul(attention, V)
pd.DataFrame(new_V)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.052224,0.030378,0.095681,0.094239,0.123860,0.113760,0.002573,0.146985,0.003605,0.055272,...,0.139466,0.095609,0.071691,0.116585,0.094919,0.118280,0.157386,0.036496,0.008493,0.168269
1,0.151522,0.085242,0.174118,0.103466,0.135389,0.164804,0.048483,0.283529,0.006363,0.208102,...,0.253220,0.192097,0.130385,0.172809,0.214880,0.181397,0.249064,0.046052,0.034634,0.197303
2,0.214962,0.187481,0.214652,0.233944,0.220438,0.262651,0.169419,0.348725,0.066204,0.322687,...,0.400753,0.247167,0.192598,0.200867,0.330464,0.225413,0.287572,0.062315,0.054945,0.253328
